# Homework 2 

## ECS 271 

## Kay Royo


### 1.) 

Implement a single-layer fully connected neural network model to classify MNIST images. It inputs
a raw image as a 1D vector of length 784=28x28 and outputs a vector of length 10 (each dimension
corresponds to a class). You may want to add a bias term to the input, but that is optional for this
assignment. The output is connected to the first layer simply by a set of linear weights. The second
layer uses Softmax function as the non-linearity function. Softmax is a simple function that converts a
n-dimensional input (z) to a n-dimensional output (o) where the output sums to one and each element
is a value between 0 and 1. It is defined as

$$y_i = \frac{exp(x_i)}{\sum_{j=1}^n exp(x_j)}$$

When we apply this function to the output of the network, $o$, it predicts a vector which can be seen as
the probability of each category given the input $x$:

$$P(c_i|x) = \frac{exp(o_i)}{\sum^n_{j=1} exp(o_j )}$$


where $n$ is the number of categories, 10, in our case. We want the $i$’th output to mimic $P(c_i|x)$,
the probability of the input $x$ belonging to the category $i$. We can represent the desired probability
distribution as the vector $gt$ where $gt(i)$ is one only if the input is from the $i$’th category and zero
otherwise. This is called one-hot encoding. Assuming $x$ is from $y$’th category, $gt(y)$ is the only element
in $gt$ that is equal to one. Then, we want the output probability distribution to be similar to the desired
one (ground-truth). Hence, we use cross-entropy loss to compare these two probability distributions,
$P$ and $gt$:

$$L(x,y,w) = \sum_{i=1}^n - gt(i) log(P(c_i|x))$$

where $n$ is the number of categories. Since $gt$ is one hot encoding, we can remove the terms of $gt$ that
are zero, keeping only the $y$’th term. Since $gt(y) = 1$, we can remove it in the multiplication to come
up with the following loss which is identical to the above one:

$$L(x,y,w) = - log(P(c_y|x))$$

This is the loss for one input only, so the total loss on a mini-batch is:

$$L = \sum_{k=1}^N -log(P(c_{yk}|x_k)) $$

where $N$ is the size of mini-batch, number of training data being processed at this iteration.

Please implement stochastic gradient descend (SGD) algorithm from scratch to train the model. You
may use NumPy, but should not use PyTorch, TensorFlow, or any similar deep learning framework.
Use mini-batch of 10 images per iteration. Then, train it on all MNIST train dataset and plot the
accuracy on all test data for every n iteration. Please choose n small enough so that the graph shows
the progress of learning and large enough so that testing does not take a lot of time. You may use
smaller n initially and then increase it gradually as the learning progresses. Choose a learning rate so
that the loss goes down.

*Answer:*

(TODO: easiest way to get the gradient of loss w.r.t. weights will be to derive the derivative manually on paper to get the final formula (probably using chain rule manually) and then write a function that calculates the formula in Python. Please include you derivation for the formula in you report by typing it in Latex)

In [1]:
# Import dependencies 
import pandas as pd 
import nbconvert
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import operator 
from operator import itemgetter
import plotly.express as px
import timeit

In [2]:
#Load dataset 
M = loadmat('..\HW1\data\MNIST_digit_data.mat')
images_train,images_test,labels_train,labels_test= M['images_train'],M['images_test'],M['labels_train'],M['labels_test']

np.random.seed(1)

#randomly permute data points
inds = np.random.permutation(images_train.shape[0])
images_train = images_train[inds]
labels_train = labels_train[inds]

inds = np.random.permutation(images_test.shape[0])
images_test = images_test[inds]
labels_test = labels_test[inds]

print(images_train.shape, labels_train.shape)
print(images_test.shape, labels_test.shape)

(60000, 784) (60000, 1)
(10000, 784) (10000, 1)


In [17]:
images_train[0].shape

(784,)

In [11]:
#one hot encoding function 
def one_hot_enc(Y):
    t = np.zeros((Y.shape[0], 10))
    for i in range(Y.shape[0]):
        t[i][int(Y[i][0])] = 1 
    return t

#normalization function 
def normalize(X): 
    X = X / 255
    return X 

images_train_norm = normalize(images_train)
labels_train = one_hot_enc(labels_train)
images_test_norm = normalize(images_test)
labels_test = one_hot_enc(labels_test)

In [16]:
images_train_norm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [80]:
#stochastic gradient descent algorithm
class neural_net: 
    #initialize empty layer dicts 
    l1 = {} #matrix of size (784,10)
    l2 = {} #matrix of size(10,10)
        
    #initialize model parameters, such as 1st & 2nd layer parameters and biases
    def __init__(self, inputs, hidden, outputs):
        self.input_size = inputs
        self.hidden_size = hidden
        self.output_size = outputs
        self.l1['params'] = np.random.randn(inputs, hidden) / np.sqrt(hidden) #layer1 (normalized)
        self.l1['bias'] = np.random.randn(hidden,1) / np.sqrt(hidden) #include bias 
        self.l2['params'] = np.random.randn(hidden,outputs) / np.sqrt(hidden) #layer2 (normalized)
        self.l2['bias'] = np.random.randn(outputs,1) / np.sqrt(hidden) #include bias
    
    #activation functions 
    def softmax(self, x):
    #simplified to prevent overflow
        exp = np.exp(x - x.max())
        return exp / np.sum(exp, axis=0)
    
    def dsoftmax(self, X):
        #derivative of softmax
        exp=np.exp(X-X.max())
        return exp/np.sum(exp,axis=0)*(1-exp/np.sum(exp,axis=0))

    def relu(self, X):
        return np.maximum(0,X)

    def drelu(self, X):
        #derivative of relu 
        return 1 * (X > 0) 
    
    #cross entropy error
    def cross_entropy_err(self, y,y_p):
        loss = -np.sum(y*np.log(y_p))
        return loss/float(y_p.shape[0])
    
    #forward pass 
    def forward(self,X,Y):
        # input layer to l1 
        X_l1 = np.matmul((self.l1['params']),X) + self.l1['bias']
        X_relu = np.array(self.relu(X_l1))
        # l1 to output layer
        X_l2 = np.matmul(self.l2['params'],X_relu).reshape((self.output_size,1)) + self.l2['bias']
        out_put = np.squeeze(self.softmax(X_l2)) #compute predicted list using softmax 
        #compute error 
        error = self.cross_entropy_err(Y,out_put) 
        #save results in dictionary 
        dict_f = {'X_l1':X_l1, 'X_relu':X_relu, 'X_l2':X_l2, 'Y_pred':out_put.reshape((1,self.output_size)), 'error':error }
        
        return dict_f 
    
    #back propagation   
    def backward(self,X,Y,F):
        #one-hot encoded labels
        targets = np.array([0]*self.output_size).reshape((1,self.output_size))
        targets[0][Y] = 1
        
        #compute l2 update 
        err2 = 2*(F['Y_pred']-targets)/F['Y_pred'].shape[0]*self.dsoftmax(F['X_l2'])
        l2_update = np.matmul(err2,F['X_relu'])
        
        #compute l1 update
        err1 = (np.matmul(self.l2['params'].transpose(),err2)).transpose()*self.drelu(F['X_l1'])
        l1_update = err1*self.relu(F['X_l1'])

        dict_b = {
            'err2':err2,
            'l2_update':l2_update,
            'err1':err1,
            'l1_update':l1_update
        }
        return dict_b
    
    #loss function of training set
    def loss_func(self,X_tr,Y_tr):
        loss = 0
        for n in range(len(X_train)):
            y = Y_tr[n]
            x = X_tr[n][:]
            loss += self.forward(x,y)['error']
        return loss
    
    #training
    def train(self, X_tr, Y_tr, n = 1000, batch = 10, learning_rate = 0.01):
        
        test_dict = {} 
        loss_dict = {}
        count = 1
        for i in range(n): 
            # generate a random indices for the training set
            indices = np.random.choice(len(X_tr), batch, replace=True)
            F = self.forward(X_tr[indices],Y_tr[indices])
            B = self.backward(X_tr[indices],Y_tr[indices],F)
            #update hyperparams
            self.l1['params'] -= learning_rate*B['l1_update']
            self.l1['bias'] -= learning_rate*B['err1']
            self.l2['params'] -= learning_rate*B['l2_update']
            self.l2['bias'] -= learning_rate*B['err2']

            if count % 1000 == 0:
                    loss = self.loss_func(X_tr,Y_tr)
                    test = self.test(x_test,y_test)
                    print('Trained for {} times,'.format(count),'loss = {}, test = {}'.format(loss,test))
                    loss_dict[str(count)]=loss
                    test_dict[str(count)]=test
            count += 1
        print('Training completed.')
        return loss_dict, test_dict
    
    #testing 
    def test(self,X_te, Y_te):
        correct = 0
        for n in range(len(X_te)):
            x = X_te[n][:]
            y = Y_te[n]
            pred = np.argmax(self.forward(x,y)['Y_pred'])
            for n in range(len(y))
            if (prediction == y):
                correct += 1 #sum correctly identified labels 
        print('Test Accuarcy: ',correct/len(X_te))
        return correct/np.float(len(X_te))

In [81]:
n= 20000 #number of iterations
lr = 0.01 #learning rate 
batch = 10 #batch size 
input_size = 28*28 #num inputs
output_size = 10 #num outputs
hidden_size = 10 #hidden layer size 

In [82]:
model = neural_net(input_size,hidden_size,output_size)

In [83]:
loss_dict, test_dict = model.train(images_train,labels_train,n=n,batch=batch, learning_rate=lr)

ValueError: operands could not be broadcast together with shapes (784,784) (10,1) 

In [79]:
l1 = {}
l1['params'] = np.random.randn(input_size, hidden_size) / np.sqrt(input_size)
l1['params'].shape

(784, 10)

In [24]:
def init(x,y):
    layer=np.random.uniform(-1.,1.,size=(x,y))/np.sqrt(x*y)
    return layer.astype(np.float32)

np.random.seed(42)
l1=init(28*28,128)
l2=init(128,10)

In [25]:
l1

array([[-7.9208513e-04,  2.8455637e-03,  1.4646830e-03, ...,
        -4.6020158e-04,  2.0077715e-03,  2.2774558e-03],
       [-3.1128346e-03,  6.7852598e-05, -5.2142178e-04, ...,
        -1.1558544e-03, -2.0866422e-03,  3.5861213e-04],
       [ 2.7536429e-03,  1.2376250e-03,  4.4232793e-04, ...,
         1.2420909e-03,  8.1407465e-04,  2.3831520e-03],
       ...,
       [-4.8830349e-05,  6.0965895e-04, -2.5888244e-03, ...,
        -2.5404603e-03,  3.0673348e-04,  5.6949357e-04],
       [ 4.0616604e-04,  2.9352377e-03, -1.1387233e-03, ...,
         2.8009422e-03, -2.8184371e-03,  1.9042534e-04],
       [-3.7994463e-04,  1.5051493e-03, -1.2513590e-03, ...,
         1.4320496e-03,  1.6163822e-03, -1.6940964e-03]], dtype=float32)

In [12]:
#stochastic (mini-batch) gradient descent algorithm


In [ ]:
#Plot the accuracy on all test data for every n iteration

### 2.) 

For each class, visualize the 10 images that are misclassified with the highest score along with their
predicted label and score. These are very confident wrong predictions.


### 3.) 

Please reduce the number of training data to 1 example per class (chosen randomly from training data)
and plot the curve (accuracy vs, iterations). The whole training data will be 10 images only.


### 4.) 

Try different mini-batch sizes (1, 10, 100) for the original case and plot the results. Which one is better
and why?


### 5.) 

Instead of using random sampling, sort the data before training so that all ”1”s appear before ”2”s
and so on. Then, sample sequentially in running SGD instead of random sampling. Does this work
well, why?


### 6.) 

(Bonus point) Add a hidden layer with 11 hidden neurons and ReLU activation function. Then, plot
the accuracy curve to see how the accuracy changes.

Note: if you want to solve the bonus question since there are two more layers (a linear layer and a ReLU activation layer). If you want to do the bonus too, you may want to code back-propagation by calculating the derivative formula for each layer and hard-coding it into your python code. Then, you can write code that multiplies them to get the final gradient as discussed in the class. If you go this route, you can use it for both bonus and non-bonus parts of the assignment